# Aurora Cloning for Testing and Development

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Setting-up-the-environment-and-required-libraries">Setting up the environment and required libraries</a></li>
<li><a href="#1-Creating-Database-Clones">1. Creating Database Clones</a></li>
<li><a href="#2-Using-Clones-for-Testing-and-Development">2. Using Clones for Testing and Development</a></li>
<li><a href="#3-Best-Practices-for-Managing-Clones">3. Best Practices for Managing Clones</a></li>
<li><a href="#4-Stress-Test-Strategies-and-Tools">4. Stress Test Strategies and Tools</a></li>
</ul>
</div>

## Prerequisites

Before starting this workshop, ensure you have:

- ✅ **Jupyter Notebook**: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
- ✅ **Aurora PostgreSQL Cluster**: An active Aurora PostgreSQL cluster running in your AWS account
  - If you don't have one, follow the setup guide: [Your First Database on AWS](../../2_Your_First_Database_on_AWS/README.MD)
- ✅ **AWS CLI configured** with appropriate permissions for RDS operations
- ✅ **IAM permissions** for:
  - Aurora cluster cloning and restoration
  - RDS instance creation and management
  - Resource tagging and cleanup operations

> 💡 **Note**: Aurora cloning uses copy-on-write technology, making it fast and cost-effective for testing environments.

## Setting up the environment and required libraries

This step initializes the required shell parameters to proceed with next steps:

In [ ]:
%%bash

echo "Setting parameters"
source_cluster_id=<source-cluster-identifier>
clone_cluster_id=<clone-cluster-identifier>

## 1. Creating Database Clones

These functions create Aurora clones using copy-on-write technology for fast, cost-effective database copies and monitor the cloning process until completion. Aurora cloning creates a new cluster that shares the same underlying storage as the source, making it ideal for testing and development. You'll see clone creation responses and availability status updates.

In [ ]:
%%bash
export source_cluster_id=<source-cluster-identifier>
export clone_cluster_id=<clone-cluster-identifier>

echo "Creating a Database Clone"
aws rds restore-db-cluster-to-point-in-time \
    --db-cluster-identifier $clone_cluster_id \
    --source-db-cluster-identifier $source_cluster_id \
    --restore-type copy-on-write \
    --use-latest-restorable-time \
    --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8

echo "✅ Cluster Cloned Successfully"

## 2. Using Clones for Testing and Development

This function configures Aurora clones specifically for testing environments by creating database instances and applying test-optimized parameter settings. The setup includes performance tuning parameters suitable for development workloads and testing scenarios. You'll see instance creation responses and parameter group modifications for optimal testing performance.

In [ ]:
%%bash
export source_cluster_id=<source-cluster-identifier>
export clone_cluster_id=<clone-cluster-identifier>

echo "Step 1: Create DB instance in the clone cluster"
aws rds create-db-instance \
    --db-instance-identifier "${clone_cluster_id}-instance-1" \
    --db-cluster-identifier ${clone_cluster_id} \
    --engine aurora-postgresql \
    --db-instance-class db.r5.large \
    --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8

echo "Step 2: Get the parameter group name from the created instance"
parameter_group_name=$(aws rds describe-db-instances \
    --db-instance-identifier "${clone_cluster_id}-instance-1" \
    --query 'DBInstances[0].DBParameterGroups[0].DBParameterGroupName' \
    --output text)

echo "Step 3: Modify the parameter group with new values"
aws rds modify-db-parameter-group \
    --db-parameter-group-name ${parameter_group_name} \
    --parameters \
    "ParameterName=max_connections,ParameterValue=1000,ApplyMethod=immediate" \
    "ParameterName=work_mem,ParameterValue=32768,ApplyMethod=immediate"

echo "✅ Instance Node Created and Configured Successfully"

## 3. Best Practices for Managing Clones

These functions implement clone lifecycle management including automated cleanup of old clones and proper resource tagging for cost control and organization. The management system prevents clone sprawl by automatically removing outdated test environments and ensures proper resource tracking. You'll see cleanup operations and tagging confirmations for effective clone governance.

In [ ]:
%%bash

export source_cluster_id=<source-cluster-identifier>
export clone_cluster_id=<clone-cluster-identifier>

clone_cluster_id=$1
current_date=$(date +%Y-%m-%d)
deletion_threshold=7  # days

# Function to clean up old clones
cleanup_old_clones() {
    echo "Cleaning up old clones..."
    
    # Get all DB clusters
    clusters=$(aws rds describe-db-clusters --query 'DBClusters[*].[DBClusterIdentifier,ClusterCreateTime]' --output text)
    
    while IFS=$'\t' read -r cluster_id create_time; do
        # Check if cluster ID starts with 'clone-'
        if [[ $cluster_id == clone-* ]]; then
            # Convert creation date to seconds since epoch
            create_date_seconds=$(date -d "$create_time" +%s)
            current_date_seconds=$(date +%s)
            
            # Calculate age in days
            age_days=$(( ($current_date_seconds - $create_date_seconds) / 86400 ))
            
            # Delete if older than threshold
            if [ $age_days -gt $deletion_threshold ]; then
                echo "Deleting old clone: $cluster_id (age: $age_days days)"
                aws rds delete-db-cluster \
                    --db-cluster-identifier "$cluster_id" \
                    --skip-final-snapshot
            fi
        fi
    done <<< "$clusters"
}

# Function to tag clone resources
tag_clone_resources() {
    echo "Tagging clone resources..."
    
    # Get the ARN of the cluster
    cluster_arn=$(aws rds describe-db-clusters \
        --db-cluster-identifier "$clone_cluster_id" \
        --query 'DBClusters[0].DBClusterArn' \
        --output text)
    
    # Add tags to the cluster
    aws rds add-tags-to-resource \
        --resource-name "$cluster_arn" \
        --tags \
        "Key=Environment,Value=Test" \
        "Key=CreatedDate,Value=$current_date" \
        "Key=AutoDelete,Value=True" \
        "Key=CreationSource,Value=aws-database-cookbook-v2025.8"
}

# Main execution
if [ -z "$clone_cluster_id" ]; then
    echo "Error: Clone cluster ID not provided"
    echo "Usage: $0 <clone-cluster-id>"
    exit 1
fi

# Execute the functions
cleanup_old_clones
tag_clone_resources

echo "Clone management completed successfully"

## Next Steps 🚀

Congratulations! You've completed the Operational Excellence section and mastered Aurora cloning for testing and development. Now it's time to scale your Aurora infrastructure:

1. **Scale Your Aurora Infrastructure**
   - Learn advanced scaling strategies in [Section 5: Scaling for Success - Growing with Aurora](../../5_Scaling_for_Success_Growing_with_Aurora/README.md)
   - Implement vertical and horizontal scaling techniques

2. **Implement Read Replicas**
   - Set up read replicas for load distribution in [5.2 Implementing Read Replicas](../../5_Scaling_for_Success_Growing_with_Aurora/5.2_Implementing_Read_Replicas/README.md)
   - Learn connection routing and load balancing

3. **Advanced Scaling Strategies**
   - Explore Aurora Serverless v2 and auto-scaling features
   - Implement global database deployments for worldwide scale

Continue to [Section 5: Scaling for Success - Growing with Aurora](../../5_Scaling_for_Success_Growing_with_Aurora/README.md) to learn how to scale your Aurora infrastructure for production workloads!


## Additional Resources 📚

### Aurora Cloning
- [Aurora Cloning Documentation](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Managing.Clone.html)
- [Aurora Clone Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.html)
- [Copy-on-Write Technology](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Managing.Clone.html#Aurora.Managing.Clone.Protocol)

### Testing & Development
- [Database Testing Strategies](https://docs.aws.amazon.com/wellarchitected/latest/reliability-pillar/test-reliability.html)
- [Aurora Performance Testing](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Managing.Performance.html)
- [Development Environment Best Practices](https://docs.aws.amazon.com/whitepapers/latest/development-test-on-aws/development-test-on-aws.html)

### Cost Management
- [Aurora Cost Optimization](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Managing.html)
- [Resource Tagging Strategies](https://docs.aws.amazon.com/general/latest/gr/aws_tagging.html)
- [AWS Cost Management](https://docs.aws.amazon.com/awsaccountbilling/latest/aboutv2/cost-alloc-tags.html)